# STC Jawwy

In [1]:
"""
Here we install libraries that are not installed by default 
Example:  pyslsb
Feel free to add any library you are planning to use.
"""
!pip install pyxlsb

In [2]:
# Import the required libraries 
"""
Please feel free to import any required libraries as per your needs
"""
import pandas as pd     # provides high-performance, easy to use structures and data analysis tools
import pyxlsb           # Excel extention to read xlsb files (the input file)
import numpy as np      # provides fast mathematical computation on arrays and matrices

# Jawwy dataset
The dataset consists of details about each customer and the movies and/or tv shows watched in addition to the genre. 

You are required to work on task three to build a recommendation engine for our platform to Recommend movies to usesrs that they might be interested in¶


In [3]:
dataframe = pd.read_excel(r"C:\Users\Thinkpad\Desktop\STC\Task 3\stc TV Data Set_T3.xlsb",index_col=0)
# Please make a copy of dataset if you are going to work directly and make changes on the dataset
# you can use   df=dataframe.copy()

In [4]:
# check the data shape
dataframe.shape

(1048575, 5)

In [5]:
# display the first 5 rows 
dataframe.head()

,user_id_maped,program_name,rating,date_,program_genre
0,26138,100 treets,1,2017-05-27,Drama
1,7946,Moana,1,2017-05-21,Animation
2,7418,The Mermaid Princess,1,2017-08-10,Animation
3,19307,The Mermaid Princess,2,2017-07-26,Animation
4,15860,Churchill,2,2017-07-07,Biography


In [6]:
# describe the numeric values in the dataset
dataframe.describe()

,user_id_maped,rating,date_
count,1.048575e+06,1.048575e+06,1048575
mean,1.709266e+04,2.497283e+00,2017-10-04 00:23:20.346183936
min,1.000000e+00,1.000000e+00,2017-03-14 00:00:00
25%,8.253000e+03,1.000000e+00,2017-06-10 00:00:00
50%,1.714900e+04,2.000000e+00,2017-10-14 00:00:00
75%,2.566500e+04,3.000000e+00,2018-01-21 00:00:00
max,3.428000e+04,4.000000e+00,2018-04-30 00:00:00
std,1.003513e+04,1.119837e+00,NaN


In [7]:
# check if any column has null value in the dataset
dataframe.isnull().any()

user_id_maped    False
program_name     False
rating           False
date_            False
program_genre    False
dtype: bool

In [8]:
# we import Visualization libraries 
# you can ignore and use any other graphing libraries 
import matplotlib.pyplot as plt # a comprehensive library for creating static, animated, and interactive visualizations
import plotly #a graphing library makes interactive, publication-quality graphs. Examples of how to make line plots, scatter plots, area charts, bar charts, error bars, box plots, histograms, heatmaps, subplots, multiple-axes, polar charts, and bubble charts.
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [9]:
"""
TODO build your Recommender system to Highlight Programs that usesrs might be interested in
"""

'\nTODO build your Recommender system to Highlight Programs that usesrs might be interested in\n'

In [10]:
"""
TODO show the recommendations (top 5) for the people who watched "Moana" movie
"""

'\nTODO show the recommendations (top 5) for the people who watched "Moana" movie\n'

In [11]:
df = dataframe.copy()

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, Dot
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError
import seaborn as sns 
sns.set_style()

import plotly.express as px

In [13]:
# Create mappings for user_id and program_name to numeric ids
user_ids = df['user_id_maped'].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}

program_names = df['program_name'].unique().tolist()
program2program_encoded = {x: i for i, x in enumerate(program_names)}
program_encoded2program = {i: x for i, x in enumerate(program_names)}


In [14]:
df['user'] = df['user_id_maped'].map(user2user_encoded)
df['program'] = df['program_name'].map(program2program_encoded)

num_users = len(user2user_encoded)
num_programs = len(program_encoded2program)


In [15]:
# Split the data into train and test
X = df[['user', 'program']].values
y = df['rating'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# Model building
user_input = Input(shape=(1,), name='user_input', dtype='int64')
user_embedding = Embedding(num_users, 15, name='user_embedding')(user_input)
user_vec = Flatten(name='FlattenUsers')(user_embedding)

program_input = Input(shape=(1,), name='program_input', dtype='int64')
program_embedding = Embedding(num_programs, 15, name='program_embedding')(program_input)
program_vec = Flatten(name='FlattenPrograms')(program_embedding)

product = Dot(name='DotProduct', axes=1)([program_vec, user_vec])
model = Model([user_input, program_input], product)
model.compile('adam', 'mean_squared_error')

history = model.fit([X_train[:,0], X_train[:,1]], y_train, epochs=10, verbose=1)

Epoch 1/3
26215/26215 [==============================] - 53s 2ms/step - loss: 2.3277
Epoch 2/3
26215/26215 [==============================] - 53s 2ms/step - loss: 0.8765
Epoch 3/3
26215/26215 [==============================] - 50s 2ms/step - loss: 0.8107


In [22]:
loss_values = history.history['loss']

# Create a line chart using Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range(1, len(loss_values) + 1)),
    y=loss_values,
    mode='lines+markers',
    name='Training Loss',
    marker=dict(size=8, color='blue'),
    line=dict(color='blue', width=2)
))

fig.update_layout(
    title='Training Loss Over Epochs',
    xaxis_title='Epochs',
    yaxis_title='Loss',
    template='plotly_dark',
    hovermode='x unified',
    legend=dict(x=0.7, y=0.9),
    font=dict(family='Arial', size=12),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
)

# Adding vertical line to indicate the best epoch (lowest loss)
best_epoch = loss_values.index(min(loss_values)) + 1
fig.add_shape(
    type="line",
    x0=best_epoch,
    x1=best_epoch,
    y0=0,
    y1=max(loss_values),
    line=dict(color="red", width=2, dash="dash"),
)
fig.add_annotation(
    x=best_epoch,
    y=max(loss_values),
    text="Best Epoch",
    showarrow=True,
    arrowhead=2,
    arrowsize=1,
    arrowcolor="red",
    ax=20,
    ay=-40,
    font=dict(color="white", size=12),
)

fig.show()

In [18]:
# Now let's make a prediction for the users who watched "Moana"
program_id = program2program_encoded["Moana"]
user_program_array = np.hstack(
    [np.array(list(user2user_encoded.values())).reshape(-1,1), np.array([program_id]*num_users).reshape(-1,1)]
)

ratings = model.predict([user_program_array[:,0], user_program_array[:,1]]).flatten()

top_users_indices = ratings.argsort()[-5:][::-1]
recommended_users = [userencoded2user.get(users) for users in top_users_indices]

print(recommended_users)

362/362 [==============================] - 0s 578us/step
[28810, 4790, 28069, 4859, 5942]


In [35]:
# Visualize predicted ratings for recommended users
fig_predicted_ratings = px.bar(
    x=recommended_users,
    y=ratings[top_users_indices],
    title='Predicted Ratings for Top Recommended Users',
    labels={'x': 'Recommended Users', 'y': 'Predicted Rating'},
    text=ratings[top_users_indices].round(2),  # Display rating values on the bars
)

# Customize the bar appearance
fig_predicted_ratings.update_traces(
    marker_color='rgb(158, 202, 225)',
    marker_line_color='rgb(8, 48, 107)',
    marker_line_width=1.5,
    opacity=0.6,
    textposition='outside',   # Position the text labels outside the bars
)

# Update layout for a clean and professional look
fig_predicted_ratings.update_layout(
    xaxis_title='Recommended Users',
    yaxis_title='Predicted Rating',
    title_x=0.5,  # Center the title
    showlegend=False,  # Hide legends since it's clear from the title
)

# Display the figure
fig_predicted_ratings.show()